In [1]:
import requests
import json
import sys
import pandas as pd
import numpy as np

from  keys  import  client_id, api_key
#import Yelp_Helper_Functions as yf

Set the URL and headers for authorization including api key

In [2]:
# set the url and endpoint we want to hit
url        = 'https://api.yelp.com/v3/businesses/search'
headers    = {
             'Authorization': 'Bearer {}'.format(api_key),
             }
# what type of business do you want to search
term       = 'bars'
# where do you want to perform this search
location   = 'Denver'
# what is your other parameter you want to search against
categories = 'restaurants'
# set url parameters
url_params = {
                "term"       : term.replace(' ', '+'),
                "location"   : location.replace(' ', '+'),
                "categories" : categories,
                "limit"      : 50,
                "offset"     : 0
              }
response   = requests.get(url, headers=headers, params=url_params)

In [3]:
# call yelp api
def yelp_call(url_params, api_key):
    # use .get() to call url and set to 'response'
    response = requests.get(url, headers=headers, params=url_params)
    # return json file from yelp api for all businesses
    return response.json()['businesses']

In [4]:
# parse yelp data according to parameters
def parse_data(list_of_data):
    # create a container to hold our parsed data
    businesses = []
    # loop through our business  
    for business in list_of_data:
        # if 'price' is missing add nan value
        if 'price' not in business.keys():
            business['price'] = np.nan
        if 'delivery' in business['transactions']:
            business['delivery'] = True
        else:
            business['delivery'] = False
        # parse each individual business into a tuple
        biz_tuple = (business['id'],
                     business['name'],
                     business['location']['zip_code'],
                     business['rating'],
                     business['review_count'],
                     business['categories'][0]['alias'],
                     business['delivery'],
                     business['distance'],
                     business['price']
                    )
        # add each individual business tuple to our data container
        businesses.append(biz_tuple)
    # return the container with all of the parsed results
    return businesses

In [5]:
# open the csv file, concat the current data, and save the data.
def df_save(yb_data, parse_data):
    # concat results into previous DataFrame
    yb_data = pd.concat([yb_data, parse_data], axis='rows')  
    # save file as csv
    yb_data.to_csv("Bars_Denver.csv",
                   mode='a', index=False, header=False)
    return

In [6]:
# Python file making API call to Yelp and storing results in 'yb_data.csv'
# create a variable  to keep track of which result you are in. 
cur     = 0
num     = 1000
# create empty DataFrame called yb_data
yb_data = []
yb_data = pd.DataFrame(yb_data)
# write empty DataFrame to csv
yb_data.to_csv("Bars_Denver.csv", 
               index_label=['id', 'Name', 'Zip Code', 
                            'Buisness Rating', 'Review Count',
                            'Categories', 'Delivery', 
                            'Distance', 'Price'])
# set up a while loop to go through and grab the result 
while cur < num :
    # set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    # make your API call with the new offset number
    results     = yelp_call(url_params, api_key)
    # after you get your results you can now use your function to parse those results
    parsed_data = parse_data(results)
    # convert parsed results into a DataFrame
    parsed_data = pd.DataFrame(parsed_data)
    # use your function to insert your parsed results into the csv
    df_save(yb_data, parsed_data)
    # increment the counter by 50 to move on to the next results
    cur += 50